# Evrişimsel Sinir Ağı Oluşturma 


1. Adım: Veri Kümesini Yükleme
2. Adım: Veri Kümesini Tekrarlanabilir Yapıya Dönüştürme
3. Adım: Model Sınıfı Oluşturma
4. Adım: Model Sınıfını Oluşturma
5. Adım: Loss Sınıfını Oluşturma
6. Adım: Optimizer Sınıfını Oluşturma
7. Adım: Modelin Eğitilmesi

Bu çalışmamda Evrişimsel Sinir Ağını 2 tane Convolutional Layer, 2 tane Max Pooling Layer ve 1 tane Fully Connectted Layer ile oluşturdum.

# Adım 1: MNIST Veri Kümesini Yükleme

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_dataset = datasets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

In [3]:
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [4]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


In [5]:
print(train_dataset.train_labels.size())

torch.Size([60000])


In [6]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


In [7]:
print(test_dataset.test_labels.size())

torch.Size([10000])


## 2. Adım: Veri Kümesini Tekrarlanabilir Yapıya Dönüştürme 

In [8]:
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters/(len(train_dataset) / batch_size))


In [9]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle = True)

In [10]:
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle = True)

## 3. Adım: Model Sınıfı Oluşturma


In [11]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        # Comvolutaion Layer 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size = 5, stride = 1, padding = 2)
        self.relu1 = nn.ReLU()
        
        # Max Pooling Layer 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Comvolutaion Layer 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size = 5, stride = 1, padding = 2)
        self.relu2 = nn.ReLU()
        
        # Max Pooling Layer 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected Layer 1
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        
        # Convulation Layer 1
        x = self.cnn1(x)
        x = self.relu1(x)
        
        # Max Pooling 1
        x = self.maxpool1(x)
        
        # Convulation Layer 2
        x = self.cnn2(x)
        x = self.relu2(x)
        
        # Max Pooling 2
        x = self.maxpool2(x)
        
        # Resize 
        out = x.view(x.size(0), -1)
        
        # Fully Connected Layer
        
        out = self.fc1(out)
        
        return out
        
        

## 4. Adım: Model Sınıfını Oluşturma


In [12]:
model = CNN_Model()

## 5. Adım: Loss Sınıfını Oluşturma


In [13]:
criterion = nn.CrossEntropyLoss()

## 6. Adım: Optimizer Sınıfını Oluşturma


Her iterasyonda modelimizin parametrelerini güncelliyoruz.

$\theta = \theta - \eta \cdot \nabla_\theta $

$\theta$: parameters

$\eta$: learning rate 

$\nabla_\theta$: parameters' gradients


parameters = parameters - learning_rate * parameters_gradients


In [14]:
learning_rate = 0.01

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [16]:
print(model)

CNN_Model(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [17]:
print(model.parameters())

<generator object Module.parameters at 0x000002858102F0C8>


In [18]:
print(len(list(model.parameters())))

6


In [19]:
print(list(model.parameters())[0].size())

torch.Size([16, 1, 5, 5])


In [20]:
print(list(model.parameters())[1].size())

torch.Size([16])


In [21]:
print(list(model.parameters())[2].size())

torch.Size([32, 16, 5, 5])


In [22]:
print(list(model.parameters())[3].size())

torch.Size([32])


In [23]:
print(list(model.parameters())[4].size())

torch.Size([10, 1568])


In [24]:
print(list(model.parameters())[5].size())

torch.Size([10])


# 7. Adım: Modelin Eğitilmesi

İzlenecek adımlar şu şekilde olacaktır.

* 1. Input ve labelları değişkenlere dönüştüreceğiz.
 CNN Input:(1,28,28) ve Feedforward NN Input:(1,28*28)
 
* 2. Gradyanları temizleme

* 3. Verilen inpulardan çıktı alma

* 4. Loss Hesabı

* 5. Gradyan hesaplanması

* 6. Gradyanları kullanarak parametreleri güncelleme ( parameters = parameters - learning_rate * parameters_gradients )

* 7. Tekrarlama İşlemi



In [29]:
iteration = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # image yükleme
        images = images.requires_grad_()
        
        # gradyanları sıfırlama
        optimizer.zero_grad()
        
        # ileri yayılım ile çıktı elde etme  
        outputs = model(images)
        
        # Loss Hesaplama: softmax (çapraz entropi kaybı)
        loss = criterion(outputs, labels)
        
        # geriye yayılım
        loss.backward()
        
        # parametreleri güncelleme
        optimizer.step()
        
        iteration += 1
        
        if iteration % 500 == 0:
            
            # doğruluk hesabı      
            correct = 0
            total = 0
            
            # test datası boyunca iterasyon yapma
            for images, labels in test_loader:
                # image yükleme
                images = images.requires_grad_()
                
                # ileri yayılım ile çıktı elde etme  
                outputs = model(images)
                
                # max değeri sağlayan tahmini alma 
                _, predicted = torch.max(outputs.data, 1)
                
                # labelların toplam sayısı
                total += labels.size(0)
                
                # toplam doğru tahmin
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct // total
            
            # Print Loss
            print('Iteration: {}.  ||   Loss: {}.  ||  Accuracy: {}'.format(iteration, loss.item(), accuracy))

Iteration: 500.  ||   Loss: 0.07009396702051163.  ||  Accuracy: 97
Iteration: 1000.  ||   Loss: 0.039620377123355865.  ||  Accuracy: 97
Iteration: 1500.  ||   Loss: 0.13273043930530548.  ||  Accuracy: 97
Iteration: 2000.  ||   Loss: 0.042526811361312866.  ||  Accuracy: 97
Iteration: 2500.  ||   Loss: 0.03482230007648468.  ||  Accuracy: 98
Iteration: 3000.  ||   Loss: 0.018919197842478752.  ||  Accuracy: 98
